In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vvkar\\OneDrive\\Desktop\\major project\\Abstractive-Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vvkar\\OneDrive\\Desktop\\major project\\Abstractive-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from Abstractive_Text_Summarizer.constants import *
from Abstractive_Text_Summarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm


[2025-01-18 09:46:40,048: INFO: config: PyTorch version 2.4.1 available.]


In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, rouge_metric, bleu_metric ,model, tokenizer, 
                               batch_size=4, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            
            
            rouge_metric.add_batch(predictions=decoded_summaries, references=target_batch)
             
            tokenized_references = [[ref.split()] for ref in target_batch]  # BLEU expects list of lists
            tokenized_predictions = [pred.split() for pred in decoded_summaries]
            bleu_metric.add_batch(predictions=tokenized_predictions, references=tokenized_references)

        #  Finally compute and return the ROUGE scores.
        rouge_score = rouge_metric.compute()
        bleu_score = bleu_metric.compute()
        return rouge_score,bleu_score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = evaluate.load('rouge')
        bleu_metric = evaluate.load('bleu')

        rouge_score, bleu_score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric ,bleu_metric, model_pegasus, tokenizer, batch_size = 4, column_text = 'dialogue', column_summary= 'summary'
            )
        
    
    
        rouge_dict = {rn: rouge_score[rn] for rn in rouge_names }
        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)
        print("RougeScores:")
        print(df)
        print("\nBleu Score:")
        print("{bleu_score['bleu']:.4f}")

In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2025-01-18 10:05:51,991: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-18 10:05:52,006: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-18 10:05:52,013: INFO: common: created directory at: artifacts]
[2025-01-18 10:05:52,018: INFO: common: created directory at: artifacts/model_evaluation]


  0%|          | 0/5 [00:24<?, ?it/s]


ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['Amanda:', 'Ask', 'Larry', ..., 'Just', 'text', 'him.'],
Input references: [['Hannah', 'needs', "Betty's", 'number', 'but', 'Amanda', "doesn't", 'have', 'it.', 'She', 'needs', 'to', 'contact', 'Larry.']]